In [2]:
import pandas as pd
import numpy as np
import random
import joblib
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('../../final/data/dataset_with_inconclusive.csv')
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,6.918734,74.271409,0,NaN,441.651953,0.508604,86.888802,NaN,NaN,NaN,NaN,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.999210,70.112738,0,NaN,414.333999,0.784442,37.659905,NaN,NaN,NaN,NaN,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,6.273777,87.682835,1,2.705387,350.319553,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,3.773108,39.425050,0,NaN,353.517996,1.569724,26.393979,NaN,NaN,NaN,NaN,Hemolytic anemia
4,11.395680,NaN,NaN,1.320481,318.250045,102.206351,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vitamin B12/Folate deficiency anemia


In [4]:
utils.get_dt_performance(df)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

(0.9993571428571428,
 0.9993908292844549,
 0.999634175387245,
 datetime.timedelta(microseconds=1985))

#### Adding correlated features

In [5]:
df['hematocrit'] = 3*df['hemoglobin']
df['rbc'] = (30*df['hemoglobin'])/df['mcv']
df['tsat'] = (df['serum_iron']/df['tibc'])*100
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,...,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label,hematocrit,tsat
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,4.604753,74.271409,0,...,441.651953,0.508604,86.888802,NaN,NaN,NaN,NaN,Anemia of chronic disease,35.627010,NaN
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.543099,70.112738,0,...,414.333999,0.784442,37.659905,NaN,NaN,NaN,NaN,Anemia of chronic disease,27.890337,NaN
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,2.912252,87.682835,1,...,350.319553,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,Vitamin B12/Folate deficiency anemia,29.341132,38.936895
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,2.828455,39.425050,0,...,353.517996,1.569724,26.393979,NaN,NaN,NaN,NaN,Hemolytic anemia,27.395064,NaN
4,11.395680,NaN,NaN,1.320481,318.250045,102.206351,NaN,3.344904,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vitamin B12/Folate deficiency anemia,34.187041,NaN


#### Dropping some columns

In [6]:
df.columns

Index(['hemoglobin', 'ferritin', 'ret_count', 'segmented_neutrophils', 'tibc',
       'mcv', 'serum_iron', 'rbc', 'age', 'gender', 'indirect_bilirubin',
       'transferrin', 'creatinine', 'cholestrol', 'copper', 'ethanol',
       'folate', 'glucose', 'label', 'hematocrit', 'tsat'],
      dtype='object')

In [7]:
cols_to_drop = ['transferrin', 'age', 'indirect_bilirubin']
df = df.drop(cols_to_drop, axis=1)
df = df[[c for c in df if c != 'label'] + ['label']]
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,4.604753,0,0.508604,86.888802,NaN,NaN,NaN,NaN,35.627010,NaN,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.543099,0,0.784442,37.659905,NaN,NaN,NaN,NaN,27.890337,NaN,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,2.912252,1,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,29.341132,38.936895,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,2.828455,0,1.569724,26.393979,NaN,NaN,NaN,NaN,27.395064,NaN,Hemolytic anemia
4,11.395680,NaN,NaN,1.320481,318.250045,102.206351,NaN,3.344904,0,NaN,NaN,NaN,NaN,NaN,NaN,34.187041,NaN,Vitamin B12/Folate deficiency anemia


#### Saving dataset 

In [8]:
df.to_csv('../../final/data/data_correlated_feats.csv', index=False)